# Hybrid model : Fuzzy CMeans and Neural Network

In [1]:
import numpy as np 
import pandas as pd 
df = pd.read_csv(r"C:\Users\Sejal Hanmante\Downloads\Phishing Detection Dataset.zip")
df.head()


,Type,url_length,number_of_dots_in_url,having_repeated_digits_in_url,number_of_digits_in_url,number_of_special_char_in_url,number_of_hyphens_in_url,number_of_underline_in_url,number_of_slash_in_url,number_of_questionmark_in_url,...,having_digits_in_subdomain,number_of_digits_in_subdomain,having_repeated_digits_in_subdomain,having_path,path_length,having_query,having_fragment,having_anchor,entropy_of_url,entropy_of_domain
0,0,37,2,0,0,8,0,0,5,0,...,0,0,1,0,3,0,0,0,4.010412,2.751629
1,1,70,5,0,0,12,0,0,6,0,...,0,0,1,0,4,0,0,0,4.089470,3.532573
2,0,42,2,0,6,8,0,0,3,1,...,0,0,1,0,1,1,0,0,4.386016,3.344698
3,0,46,2,0,0,7,0,0,4,0,...,0,0,1,0,2,0,0,0,4.221947,3.189898
4,0,51,3,0,0,9,0,0,5,0,...,0,0,1,0,3,0,0,0,4.103538,2.952820


In [8]:
pca_data=pd.read_csv(r"C:\Users\Sejal Hanmante\Downloads\pca_data.csv",index_col=[0])
pca_data.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42
0,-1.355103,0.374723,-0.847538,1.444457,-0.269435,0.582533,0.518248,0.094391,-0.305998,-0.902024,...,-0.148379,-0.215384,0.147475,0.886899,0.237549,0.741902,0.036549,0.542012,-0.363727,0.050966
1,-0.819898,-0.006518,-0.782782,-2.364139,-0.849924,0.103269,0.820576,-0.760786,0.270542,0.064134,...,-0.098172,0.407620,-0.119940,0.137893,-0.054673,-0.788291,-0.621070,0.541850,-0.126581,0.022341
2,-1.217563,0.206911,-0.533419,-0.908161,-1.798203,0.532191,0.190889,-1.048848,-0.496761,-0.296198,...,0.207842,0.083817,-0.129444,0.752606,-0.086619,1.414484,0.115978,-0.126460,-0.098412,0.042883
3,-2.460119,0.816660,-1.165692,0.991246,0.426025,0.335449,0.304665,0.919650,-0.495699,-1.303336,...,-0.220583,0.205387,0.234712,0.234143,0.039174,-0.714097,0.018968,0.273400,-0.234085,0.047169
4,-3.794874,1.397170,-1.184590,0.924645,2.939978,-0.854234,-3.340179,-2.761792,0.205402,0.789918,...,-0.575246,0.570667,-0.391723,-0.687776,-0.675614,0.519086,-0.298555,-0.460071,0.043517,0.014826


In [3]:
import skfuzzy as fuzz
import numpy as np

# Assuming your dataset is loaded into X (features of the URLs)


# Set the number of clusters (e.g., 2: phishing, benign)
n_clusters = 2

# Fuzzy C-means clustering
cntr, u, _, _, _, _, _ = fuzz.cmeans(pca_data.T, c=n_clusters, m=2, error=0.005, maxiter=1000)

# u gives the membership values of each data point in each cluster
# For each data point, you can assign membership values to your dataset


In [12]:
labels = pd.read_csv(r"C:\Users\Sejal Hanmante\OneDrive\Documents\GitHub\Phishing-url-detection\output_column.csv")
labels = labels['label'].values

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Adding FCM membership values as additional features
X_fcm = np.hstack((pca_data, u.T))  # Concatenate original features and fuzzy membership values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_fcm, labels, test_size=0.2, random_state=42)

# Scaling the features (important for NN performance)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build a simple feed-forward neural network
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification (phishing or benign)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy*100:.2f}%')


Epoch 1/10


d:\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5895/5895 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.9879 - loss: 0.0363 - val_accuracy: 0.9999 - val_loss: 0.0011
Epoch 2/10
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9998 - loss: 0.0015 - val_accuracy: 0.9999 - val_loss: 2.9491e-04
Epoch 3/10
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9998 - loss: 8.6015e-04 - val_accuracy: 0.9996 - val_loss: 9.1221e-04
Epoch 4/10
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9999 - loss: 4.9300e-04 - val_accuracy: 0.9999 - val_loss: 3.2818e-04
Epoch 5/10
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9999 - loss: 2.9387e-04 - val_accuracy: 0.9994 - val_loss: 0.0022
Epoch 6/10
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9999 - loss: 3.2865e-04 - val_accuracy: 0.9999 - val_loss: 4.6152e-04
Epoch 7/10
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9999 - loss: 1.8353e-04 - val_accuracy: 1.0000 - val_loss: 1.3717e-04
Epoch 8/10
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step -

In [14]:
import numpy as np
import skfuzzy as fuzz
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier  # Using scikit-learn's NN for easier cross-validation

# Sample data (replace with your actual features and labels)


# Perform Fuzzy C-Means Clustering
n_clusters = 2
cntr, u, _, _, _, _, _ = fuzz.cmeans(pca_data.T, c=n_clusters, m=2, error=0.005, maxiter=1000)

# Add fuzzy membership values as features
X_fcm = np.hstack((pca_data, u.T))

# Define a function to build the Neural Network (can also use other models like MLP from scikit-learn)
def build_nn():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=X_fcm.shape[1]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification (phishing or benign)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Option 1: Using K-Fold Cross-Validation with scikit-learn's MLPClassifier (easier for cross-validation)
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
fold_accuracies = []

# Iterate over each fold
for train_idx, test_idx in kf.split(X_fcm):
    X_train, X_test = X_fcm[train_idx], X_fcm[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    # Scale features for neural network
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Create and train the Neural Network model
    model = build_nn()
    model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model
    y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)  # Threshold at 0.5 for binary classification
    accuracy = accuracy_score(y_test, y_pred)
    fold_accuracies.append(accuracy)

# Calculate the average accuracy across all folds
avg_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across 5 folds: {avg_accuracy*100:.2f}%')

# Option 2: Using cross_val_score from scikit-learn with a simpler model (MLPClassifier)
# Use MLPClassifier for cross-validation
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=10, solver='adam', random_state=42)
cross_val_scores = cross_val_score(mlp, X_fcm, labels, cv=5, scoring='accuracy')

print(f'Cross-validation scores (Accuracy) for MLPClassifier: {cross_val_scores}')
print(f'Average accuracy: {np.mean(cross_val_scores)*100:.2f}%')


d:\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1474/1474 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


d:\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1474/1474 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


d:\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

d:\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1474/1474 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Average Accuracy across 5 folds: 99.99%


d:\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Cross-validation scores (Accuracy) for MLPClassifier: [0.99985157 0.99989398 0.99989398 0.99989398 0.99989398]
Average accuracy: 99.99%


d:\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
sjsbbw